In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
%matplotlib inline
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import seaborn as sns
import os
import statistics

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit

from dateutil.relativedelta import relativedelta 
from scipy.optimize import minimize              

import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product      

from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing, Holt

from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [2]:
from math import sqrt

def rmse(y_true, y_pred):
    squared_error = 0
    for i in range(len(y_true)):
        squared_error = squared_error + (y_true[i] - y_pred[i]) ** 2
    root_mean_squared_error = sqrt(squared_error / len(y_true))
    return root_mean_squared_error

In [10]:
def moving_avg(rounds):
    y_hat['avg'] = 0
    for i in range(len(Valid)):
        y_pred = 0
        for j in range(rounds):
            y_pred = y_pred + Train.footfall[len(Train) - (j + 1) * len(Valid) + i]
        y_hat['avg'][i] = y_pred / rounds

In [4]:
def test_stationary(timeseries):
    #Determine rolling statistics
    rolmean = pd.Series(timeseries).rolling(window = 28).mean()
    rolstd = pd.Series(timeseries).rolling(window = 28).std()
    
    #Plot rolling Statistics
    orig = plt.plot(timeseries, color = "blue", label = "Original")
    mean = plt.plot(rolmean, color = "red", label = "Rolling Mean")
    std = plt.plot(rolstd, color = "black", label = "Rolling Std")
    plt.legend(loc = "best")
    plt.title("Rolling Mean and Standard Deviation")
    plt.show(block = False)
    
    #Perform Dickey Fuller test
    print("Results of Dickey Fuller test: ")
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '# Lag Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print(dfoutput)

### Exclude Locations with Too Much Missing Data:

In [5]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(ff.timestamp, format = '%Y-%m-%d %H:%M:%S') 

In [6]:
ff = ff.loc[(ff.timestamp >= '2018-01') & (ff.timestamp <= '2019-08-18 23:00')]

ff_count = ff.groupby('location')['footfall'].count() / max(ff.groupby('location')['footfall'].count())
limit1 = ff_count.loc[ff_count >= 0.6].to_frame()
ff_test = ff.loc[(ff.timestamp >= '2018-07-29') & (ff.timestamp <= '2019-08-18 23:00')]
ff_test_count = ff_test.groupby('location')['footfall'].count() / max(ff_test.groupby('location')['footfall'].count())
limit2 = ff_test_count.loc[ff_test_count >= 0.8].to_frame()
limit = pd.merge(limit1, limit2, how = 'inner', on = 'location')

ff = pd.merge(ff, limit, how = 'inner', on = 'location')
ff

,Unnamed: 0,timestamp,location,device,footfall,year,month,day,hour,day_of_week,footfall_x,footfall_y
0,1344,2018-01-01 00:00:00,5.0,1780.0,494.0,2018,1,1,0,1,0.892256,0.833766
1,1345,2018-01-01 01:00:00,5.0,1780.0,899.0,2018,1,1,1,1,0.892256,0.833766
2,1346,2018-01-01 02:00:00,5.0,1780.0,770.0,2018,1,1,2,1,0.892256,0.833766
3,1347,2018-01-01 03:00:00,5.0,1780.0,599.0,2018,1,1,3,1,0.892256,0.833766
4,1348,2018-01-01 04:00:00,5.0,1780.0,331.0,2018,1,1,4,1,0.892256,0.833766
...,...,...,...,...,...,...,...,...,...,...,...,...
5002195,233635,2019-08-18 19:00:00,1168.0,827.0,42.0,2019,8,18,19,7,0.619529,0.955844
5002196,233636,2019-08-18 20:00:00,1168.0,827.0,10.0,2019,8,18,20,7,0.619529,0.955844
5002197,233637,2019-08-18 21:00:00,1168.0,827.0,13.0,2019,8,18,21,7,0.619529,0.955844
5002198,233638,2019-08-18 22:00:00,1168.0,827.0,1.0,2019,8,18,22,7,0.619529,0.955844


In [33]:
result = pd.DataFrame(columns = ('location','Naive','Moving_Average12','Moving_Average24',
                                 'Moving_Average30','Expoential_Smoothing','SARIMA'))

In [34]:
for loc in ff.location.unique()[0:10]:

    ff_loc = ff.loc[(ff.location <= loc) & (ff.location >= loc)][[
        'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_loc.index = ff_loc.timestamp
    ff_loc = ff_loc.resample('H').mean()

    ff_loc['footfall'].replace(0, np.nan, inplace = True)
    ff_loc['footfall'] = ff_loc['footfall'].inter         polate(method='linear')
    ff_loc = ff_loc.reset_index(level = ['timestamp'])
    ff_loc.timestamp = pd.to_datetime(ff_loc.timestamp, format = '%Y-%m-%d %H:%M:%S')

    temp = ff_loc[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1

    Train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')]
    Valid = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')]

    train = Train.resample('D').mean()
    valid = Valid.resample('D').mean() 

    train['footfall'] = train['footfall'].interpolate(method='linear')
    valid['footfall'] = valid['footfall'].interpolate(method='linear')

    # Monday to Thursday Ratio
    mtt = Train.loc[(Train.day_of_week >= 1) & (Train.day_of_week <= 4)].groupby('hour')['footfall'].mean().to_frame()
    mtt['ratio'] = mtt['footfall']/mtt['footfall'].sum()
    mtt = mtt.reset_index(level = ['hour'])

    # Friday Ratio
    fri = Train.loc[(Train.day_of_week >= 5) & (Train.day_of_week <= 5)].groupby('hour')['footfall'].mean().to_frame()
    fri['ratio'] = fri['footfall']/fri['footfall'].sum()
    fri = fri.reset_index(level = ['hour'])

    # Sat Ratio
    sat = Train.loc[(Train.day_of_week >= 6) & (Train.day_of_week <= 6)].groupby('hour')['footfall'].mean().to_frame()
    sat['ratio'] = sat['footfall']/sat['footfall'].sum()
    sat = sat.reset_index(level = ['hour'])

    # Sun Ratio
    sun = Train.loc[(Train.day_of_week >= 7) & (Train.day_of_week <= 7)].groupby('hour')['footfall'].mean().to_frame()
    sun['ratio'] = sun['footfall']/sun['footfall'].sum()
    sun = sun.reset_index(level = ['hour'])

    ratio1 = mtt[['hour','ratio']]
    ratio1['day_of_week'] = 1
    ratio2 = mtt[['hour','ratio']]
    ratio2['day_of_week'] = 2
    ratio3 = mtt[['hour','ratio']]
    ratio3['day_of_week'] = 3
    ratio4 = mtt[['hour','ratio']]
    ratio4['day_of_week'] = 4
    ratio5 = fri[['hour','ratio']]
    ratio5['day_of_week'] = 5
    ratio6 = sat[['hour','ratio']]
    ratio6['day_of_week'] = 6
    ratio7 = sun[['hour','ratio']]
    ratio7['day_of_week'] = 7

    ratio_week = ratio1.append(ratio2).append(ratio3).append(ratio4).append(ratio5).append(ratio6).append(ratio7)

    merge = pd.merge(Valid, ratio_week, on = ('hour','day_of_week'), how = 'left')

    # Naive Approach
    dd = np.asarray(Train['footfall'])
    y_hat = Valid.copy()
    y_hat['naive']= dd[len(dd)- 1]
    for i in range(len(Valid)): 
        y_hat['naive'][i]= dd[len(dd) - len(Valid) + i]
    naive_rmse = rmse(Valid.footfall, y_hat.naive)

    # Moving Average Approach
    moving_avg(4)
    avg4_rmse = rmse(Valid.footfall, y_hat.avg)
    moving_avg(8)
    avg8_rmse = rmse(Valid.footfall, y_hat.avg)
    moving_avg(10)
    avg10_rmse = rmse(Valid.footfall, y_hat.avg)

    # Exponential Smoothing
    y_hat = valid.copy()
    fit1 = ExponentialSmoothing(np.asarray(train.footfall), seasonal_periods = 7, trend = 'add', seasonal= 'add').fit()
    y_hat['Holt_Winter'] = fit1.forecast(len(valid))    
    HoltWinter_rmse = rmse(valid.footfall, y_hat.Holt_Winter)

    pred_d = y_hat.Holt_Winter.to_frame()
    pred_d = pred_d.reset_index(level = ['timestamp'])
    pred_d['month'] = pred_d.timestamp.dt.month
    pred_d['day'] = pred_d.timestamp.dt.day
    temp = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
    temp['prediction'] = temp['ratio'] * temp['Holt_Winter'] * 24
    holt_winter_rmse = rmse(temp['footfall'], temp['prediction'])

    # SARIMA
    fit1 = sm.tsa.statespace.SARIMAX(train.footfall, order = (1,1,3), seasonal_order =(1,1,3,7)).fit()
    y_hat['SARIMA'] = fit1.predict(start = "2019-07-29 00:00", end = "2019-08-18 23:00", dynamic=True)
    SARIMA_rmse = rmse(valid['footfall'], y_hat['SARIMA'])

    pred_d = y_hat['SARIMA'].to_frame()
    pred_d = pred_d.reset_index(level = ['timestamp'])
    pred_d['month'] = pred_d.timestamp.dt.month
    pred_d['day'] = pred_d.timestamp.dt.day
    temp2 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
    temp2['prediction'] = temp2['ratio'] * temp2['SARIMA'] * 24
    sarima_rmse = rmse(temp2['footfall'], temp2['prediction'])

    result = result.append({'location': loc, 'Naive': naive_rmse, 'Moving_Average12': avg4_rmse, 'Moving_Average24': avg8_rmse, 
                           'Moving_Average30': avg10_rmse, 'Expoential_Smoothing': holt_winter_rmse, 'SARIMA': sarima_rmse},
                           ignore_index=True)


E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:5

In [35]:
result

,location,Naive,Moving_Average12,Moving_Average24,Moving_Average30,Expoential_Smoothing,SARIMA
0,5.0,660.733618,438.124377,380.697612,435.824077,423.488557,417.842803
1,6.0,520.571453,450.796608,327.748727,371.546914,396.486996,350.549984
2,7.0,277.951764,216.414675,190.836343,208.629228,209.257630,208.223988
3,31.0,511.515018,377.816876,392.249377,440.817514,328.671785,315.860758
4,62.0,321.298819,226.211294,214.664716,248.059289,245.561794,233.264410
5,78.0,686.925250,488.953839,378.221677,440.177805,416.956271,425.775341
6,87.0,280.699218,222.816230,208.413313,209.433377,193.382826,198.950650
7,88.0,181.602965,175.427137,143.093646,142.725657,177.919970,155.171088
8,89.0,170.063667,225.046256,204.175617,211.756172,207.826243,205.387776
9,91.0,202.499036,159.444603,139.627218,143.965067,179.807262,142.644924


In [38]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

In [40]:
rt = dm_test(temp2['footfall'], temp2['prediction'], temp['prediction'],h = 1, crit="MSE")
rt

dm_return(DM=-6.931429070862556, p_value=1.2822988568989817e-11)

In [41]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [46]:
from sklearn.preprocessing import RobustScaler

In [47]:
def create_dataset(X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [52]:
result['LSTM'] = 0

In [53]:
j = 0
for loc in ff.location.unique()[0:10]:
    
    ff_loc = ff.loc[(ff.location <= loc) & (ff.location >= loc)][[
            'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_loc.index = ff_loc.timestamp
    ff_loc = ff_loc.resample('H').mean()

    ff_loc['footfall'].replace(0, np.nan, inplace = True)
    ff_loc['footfall'] = ff_loc['footfall'].interpolate(method='linear')
    ff_loc = ff_loc.reset_index(level = ['timestamp'])
    ff_loc.timestamp = pd.to_datetime(ff_loc.timestamp, format = '%Y-%m-%d %H:%M:%S')

    temp = ff_loc[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1

    train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')][[
        'footfall','month','day','hour','day_of_week']]
    test = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')][[
        'footfall','month','day','hour','day_of_week']]  
    
    ff_transformer = RobustScaler()
    ff_transformer = ff_transformer.fit(train[['footfall']])

    train['footfall'] = ff_transformer.transform(train[['footfall']])
    test['footfall'] = ff_transformer.transform(test[['footfall']])
    
    time_steps = 10
    X_train, y_train = create_dataset(train, train.footfall, time_steps)
    X_test, y_test = create_dataset(test, test.footfall, time_steps)
    
    model = keras.Sequential()
    model.add(
      keras.layers.Bidirectional(
        keras.layers.LSTM(
          units=128, 
          input_shape=(X_train.shape[1], X_train.shape[2])
        )
      )
    )
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')    
    
    history = model.fit(
    X_train, y_train, 
    epochs=30, 
    batch_size=32, 
    validation_split=0.1,
    shuffle=False
    )
    
    y_pred = model.predict(X_test)
    
    y_train_inv = ff_transformer.inverse_transform(y_train.reshape(1, -1))
    y_test_inv = ff_transformer.inverse_transform(y_test.reshape(1, -1))
    y_pred_inv = ff_transformer.inverse_transform(y_pred)
    
    lstm_rmse = rmse(y_test_inv.flatten(), y_pred_inv.flatten())
    
    result['LSTM'][j] = lstm_rmse
    
    j = j + 1

Epoch 1/30
388/388 [==============================] - 5s 13ms/step - loss: 0.2955 - val_loss: 0.3007
Epoch 2/30
388/388 [==============================] - 5s 12ms/step - loss: 0.1271 - val_loss: 0.1721
Epoch 3/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0780 - val_loss: 0.1035
Epoch 4/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0590 - val_loss: 0.0844
Epoch 5/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0478 - val_loss: 0.0671
Epoch 6/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0408 - val_loss: 0.0556
Epoch 7/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0354 - val_loss: 0.0405
Epoch 8/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0305 - val_loss: 0.0318
Epoch 9/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0277 - val_loss: 0.0299
Epoch 10/30
388/388 [==============================] - 5s 12ms/step - loss: 0.0263 - val_lo

388/388 [==============================] - 5s 13ms/step - loss: 0.0340 - val_loss: 0.0466
Epoch 22/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0327 - val_loss: 0.0442
Epoch 23/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0323 - val_loss: 0.0473
Epoch 24/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0320 - val_loss: 0.0434
Epoch 25/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0309 - val_loss: 0.0425
Epoch 26/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0303 - val_loss: 0.0404
Epoch 27/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0302 - val_loss: 0.0427
Epoch 28/30
388/388 [==============================] - 6s 16ms/step - loss: 0.0303 - val_loss: 0.0418
Epoch 29/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0292 - val_loss: 0.0442
Epoch 30/30
388/388 [==============================] - 5s 13ms/step - loss: 0.0294 - val_loss:

388/388 [==============================] - 6s 14ms/step - loss: 0.0217 - val_loss: 0.0252
Epoch 12/30
388/388 [==============================] - 6s 14ms/step - loss: 0.0215 - val_loss: 0.0255
Epoch 13/30
388/388 [==============================] - 6s 16ms/step - loss: 0.0194 - val_loss: 0.0223
Epoch 14/30
388/388 [==============================] - 6s 14ms/step - loss: 0.0203 - val_loss: 0.0228
Epoch 15/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0192 - val_loss: 0.0213
Epoch 16/30
388/388 [==============================] - 6s 16ms/step - loss: 0.0182 - val_loss: 0.0200
Epoch 17/30
388/388 [==============================] - 6s 16ms/step - loss: 0.0181 - val_loss: 0.0194
Epoch 18/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0185 - val_loss: 0.0202
Epoch 19/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0174 - val_loss: 0.0200
Epoch 20/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0171 - val_loss:

388/388 [==============================] - 6s 17ms/step - loss: 0.3908 - val_loss: 0.1361
Epoch 2/30
388/388 [==============================] - 6s 16ms/step - loss: 0.1535 - val_loss: 0.1061
Epoch 3/30
388/388 [==============================] - 7s 18ms/step - loss: 0.1105 - val_loss: 0.0884
Epoch 4/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0968 - val_loss: 0.0886
Epoch 5/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0834 - val_loss: 0.0762
Epoch 6/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0776 - val_loss: 0.0726
Epoch 7/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0699 - val_loss: 0.0707
Epoch 8/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0658 - val_loss: 0.0682
Epoch 9/30
388/388 [==============================] - 6s 15ms/step - loss: 0.0618 - val_loss: 0.0574
Epoch 10/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0574 - val_loss: 0.0530


In [54]:
result

,location,Naive,Moving_Average12,Moving_Average24,Moving_Average30,Expoential_Smoothing,SARIMA,LSTM
0,5.0,660.733618,438.124377,380.697612,435.824077,423.488557,417.842803,292
1,6.0,520.571453,450.796608,327.748727,371.546914,396.486996,350.549984,243
2,7.0,277.951764,216.414675,190.836343,208.629228,209.257630,208.223988,163
3,31.0,511.515018,377.816876,392.249377,440.817514,328.671785,315.860758,245
4,62.0,321.298819,226.211294,214.664716,248.059289,245.561794,233.264410,168
5,78.0,686.925250,488.953839,378.221677,440.177805,416.956271,425.775341,265
6,87.0,280.699218,222.816230,208.413313,209.433377,193.382826,198.950650,129
7,88.0,181.602965,175.427137,143.093646,142.725657,177.919970,155.171088,95
8,89.0,170.063667,225.046256,204.175617,211.756172,207.826243,205.387776,152
9,91.0,202.499036,159.444603,139.627218,143.965067,179.807262,142.644924,122


In [60]:
rt = dm_test(y_test_inv.flatten(), y_pred_inv.flatten(), temp2['prediction'][10:],h = 1, crit="MSE")
rt

dm_return(DM=-2.525360531691746, p_value=0.011870345175512652)